In [26]:
import urllib
import urllib2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from urlparse import urlsplit, urljoin
# Yeah I know I have scrapy above as well. I like them for different things.
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from pandas import DataFrame, Series

In [27]:
base_url = 'http://www.cra-arc.gc.ca/formspubs/t1gnrl/llyrs-eng.html'
base_page = urllib2.urlopen(base_url).read()
selection = BeautifulSoup(base_page, 'html.parser')
# print selection.prettify()

### Get a url for each year

In the middle of working on this project, they changed their website, so I had to rewrite this part.

In [28]:
uls = selection.find(property="mainContentOfPage").find_all('ul')
# Get all of the lis that have the links to individual year pages
lis = []
for ul in uls:
    for li in ul.find_all('li'):
        lis.append(li)

# Starting with 2015 and counting backwards, build up an object holding all of the urls
year_pages_dict = {}
year = 2015
for item in lis:
    year_pages_dict[str(year)] = item.a.get("href")
    year = year - 1
year_pages_dict

{'1985': u'/formspubs/prioryear/t1/1985/menu-e.html',
 '1986': u'/formspubs/prioryear/t1/1986/menu-e.html',
 '1987': u'/formspubs/prioryear/t1/1987/menu-e.html',
 '1988': u'/formspubs/prioryear/t1/1988/menu-e.html',
 '1989': u'/formspubs/prioryear/t1/1989/menu-e.html',
 '1990': u'/formspubs/prioryear/t1/1990/menu-e.html',
 '1991': u'/formspubs/prioryear/t1/1991/menu-e.html',
 '1992': u'/formspubs/prioryear/t1/1992/menu-e.html',
 '1993': u'/formspubs/prioryear/t1/1993/menu-e.html',
 '1994': u'/formspubs/prioryear/t1/1994/menu-e.html',
 '1995': u'/formspubs/prioryear/t1/1995/menu-e.html',
 '1996': u'/formspubs/prioryear/t1/1996/menu-e.html',
 '1997': u'/formspubs/prioryear/t1/1997/menu-e.html',
 '1998': u'/formspubs/prioryear/t1/1998/menu-e.html',
 '1999': u'/formspubs/prioryear/t1/1999/menu-e.html',
 '2000': u'/formspubs/prioryear/t1/2000/menu-e.html',
 '2001': u'/formspubs/prioryear/t1/2001/menu-e.html',
 '2002': u'/formspubs/prioryear/t1/2002/menu-e.html',
 '2003': u'/formspubs/priory

In [29]:
# Used to build full urls later
url_split = urlsplit(base_url)
def completeURL(partial, include_slash=False):
    if include_slash:
        return 'http://' + url_split.netloc + "/" + partial
    else:
        return 'http://' + url_split.netloc + partial

In [30]:
# Creates an object having the full urls from the partial ones
year_pages_dict_full = {}
for year, url in year_pages_dict.iteritems():
    # Create the url and then append it into the empty array
    url_joined = completeURL(url)
    year_pages_dict_full[year] = url_joined

# Debugging
year_pages_dict_full

{'1985': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1985/menu-e.html',
 '1986': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/menu-e.html',
 '1987': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1987/menu-e.html',
 '1988': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1988/menu-e.html',
 '1989': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1989/menu-e.html',
 '1990': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1990/menu-e.html',
 '1991': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1991/menu-e.html',
 '1992': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1992/menu-e.html',
 '1993': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1993/menu-e.html',
 '1994': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1994/menu-e.html',
 '1995': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1995/menu-e.html',
 '1996': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1996/menu-e.html',
 '1997': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1997/

In [31]:
provincial_links = {}

def provinceUrlListAppend(url_list, flag, year_url, year):
    print year
    if flag == "normal" or flag == "older":
        for url in url_list:
            provincial_links[year].append(completeURL(url))
    elif flag == "reallyOld":
        for url in url_list:
            link = 'http://' + url_split.netloc
            link += urlsplit(year_url).path.rsplit('/', 1)[0]
            link += "/" + url
            provincial_links[year].append(link)

def provinceUrlListProcess(url_list, year_url, year):
    provincial_links[year] = []
    # If it's 'normal' and Alberta is detected, go ahead
    if url_list[0] == 'Alberta' or url_list[0] == ' ':
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[1]/li/a/@href").extract()
        provinceUrlListAppend(url_list, "normal", year_url, year)
    else:
        # In some older links, they stuck another unordered list above
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[2]/li/a/@href").extract()
        # Another edge case check where they started using relative urls around 1998
        if url_list[0][0] == "/":
            provinceUrlListAppend(url_list, "older", year_url, year)
        else:
            provinceUrlListAppend(url_list, "reallyOld", year_url, year)

for year, year_url in year_pages_dict_full.iteritems():
    year_page = urllib2.urlopen(year_url).read()
    selection = Selector(text=year_page)
    provt_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/text()").extract()
    provinceUrlListProcess(provt_pages, year_url, year)

print provincial_links

1986
1987
1985
1988
1989
2015
2014
2011
2010
2013
2012
1991
1990
1993
1992
1995
1994
1997
1996
1999
1998
2002
2003
2000
2001
2006
2007
2004
2005
2008
2009
{'1986': [u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/alberta-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/british-columbia-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/manitoba-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/new-brunswick-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/newfoundland-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/northwest-territories-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/nova-scotia-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/ontario-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/prince-edward-island-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/quebec-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/saskatche

# Works for all provinces 1999 - 2014, except Quebec

In [32]:
def downloadPdfProvAfter99(prov_url):
    print prov_url
    # Go to the provincial page, find the doc that has the tax brackets in it
    page = urllib2.urlopen(prov_url).read()
    page_text = BeautifulSoup(page, 'html.parser')
    try: 
        download_url = completeURL(page_text.find('a', text = re.compile('428')).get("href"))

        # Go to the download page and find the pdf url
        page = urllib2.urlopen(download_url).read()
        page_text = BeautifulSoup(page, 'html.parser')
        pdf_url = completeURL(page_text.find('a', text = re.compile('pdf')).get("href"))
        pdf_url

        # Save the PDF locally
        cwd = os.getcwd()
        download_folder = os.path.join(cwd, "IncomeTaxForms")
        file_name = os.path.join(download_folder, pdf_url.split("/")[-1])

        # Speeds things up if the file already exists
        if not os.path.isfile(file_name):
            urllib.urlretrieve(pdf_url, file_name)
    except:
        print "FAILED"

In [33]:
year = "2015"
year_links = provincial_links[year]
for prov_url in year_links:
    downloadPdfProvAfter99(prov_url)

http://www.cra-arc.gc.ca/formspubs/t1gnrl/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/bc-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/mb-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/nb-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/nl-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/nt-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/ns-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/nu-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/on-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/pe-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/qc-eng.html
FAILED
http://www.cra-arc.gc.ca/formspubs/t1gnrl/sk-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/yk-eng.html
http://www.cra-arc.gc.ca/formspubs/t1gnrl/nnrsdnts-eng.html
FAILED


# Provinces for years 1995 - 1998

In [48]:
def downloadPdfProv9598(prov_url):
    print prov_url
    try:
        # Go to the provincial page, find the doc that has the tax brackets in it
        page = urllib2.urlopen(prov_url).read()
        page_text = BeautifulSoup(page, 'html.parser') 
        download_url = completeURL(page_text.find('a', text = re.compile('T1C')).get("href"))

        # Go to the download page and find the pdf url
        page = urllib2.urlopen(download_url).read()
        page_text = BeautifulSoup(page, 'html.parser')
        pdf_url = completeURL(page_text.find('a', text = re.compile('pdf')).get("href"))
        pdf_url

        # Save the PDF locally
        cwd = os.getcwd()
        download_folder = os.path.join(cwd, "IncomeTaxForms")
        file_name = os.path.join(download_folder, pdf_url.split("/")[-1])

        # Speeds things up if the file already exists
        if not os.path.isfile(file_name):
            urllib.urlretrieve(pdf_url, file_name)
    except:
        print "FAILED"

In [77]:
years = xrange(1998, 1994, -1)
for year in years:
    year_links = provincial_links[str(year)]
    for prov_url in year_links:
        downloadPdfProv9598(prov_url)

http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/british-columbia-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/manitoba-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/new-brunswick-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/newfoundland-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/northwest-territories-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/nova-scotia-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/ontario-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/prince-edward-island-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/quebec-e.html
FAILED
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/saskatchewan-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998/yukon-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1998//formspubs/prioryear/t1/1998/nonresidents-e.html
FAILED
http://www.cra

### Tried for 1994 and prior, but the tax system then looks like a gong show

# Download federal tax brackets 1999-2014

In [79]:
def downloadPdfFed(year_url):
    print year_url
    # Go to the provincial page, find the doc that has the tax brackets in it
    page = urllib2.urlopen(year_url).read()
    page_text = BeautifulSoup(page, 'html.parser')
    try: 
        download_url = completeURL(page_text.find('a', text = re.compile('Schedule 1')).get("href"))

        # Go to the download page and find the pdf url
        page = urllib2.urlopen(download_url).read()
        page_text = BeautifulSoup(page, 'html.parser')
        pdf_url = completeURL(page_text.find('a', text = re.compile('pdf')).get("href"))
        pdf_url

        # Save the PDF locally
        cwd = os.getcwd()
        download_folder = os.path.join(cwd, "IncomeTaxForms")
        file_name = os.path.join(download_folder, pdf_url.split("/")[-1])

        # Speeds things up if the file already exists
        if not os.path.isfile(file_name):
            urllib.urlretrieve(pdf_url, file_name)
    except:
        print "FAILED"

In [82]:
years = xrange(2014, 1994, -1)
for year in years:
    year_url = provincial_links[str(year)][0]
    downloadPdfFed(year_url)

http://www.cra-arc.gc.ca/formspubs/t1gnrl/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2013/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2012/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2011/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2010/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2009/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2008/ab-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2007/lbrt-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2006/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2005/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2004/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2003/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2002/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2001/alberta-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2000/alberta-e.html
http://www.cr

# Generate list of files

In [30]:
cwd = os.getcwd()
download_folder = os.path.join(cwd, "IncomeTaxForms")
import fnmatch

def all_files(root, patterns='*', single_level=False, yield_folders=False):
    # Expand patterns from semicolon-separated string to list
    patterns = patterns.split(';')
    for path, subdirs, files in os.walk(root):
        if yield_folders:
            files.extend(subdirs)
        files.sort( )
        for name in files:
            for pattern in patterns:
                if fnmatch.fnmatch(name, pattern):
                    yield name
                    break
        if single_level:
            break

file_list = DataFrame(all_files(download_folder, patterns='*.pdf'))
file_list.to_csv(download_folder + "/Filelist.csv", index=False, header=False)

In [31]:
file_list

,0
0,5000-s1-00e.pdf
1,5000-s1-01e.pdf
2,5000-s1-02e.pdf
3,5000-s1-03e.pdf
4,5000-s1-04e.pdf
5,5000-s1-05e.pdf
6,5000-s1-06e.pdf
7,5000-s1-07e.pdf
8,5000-s1-08e.pdf
9,5000-s1-09e.pdf
